In [3]:
import os
import sys
import time
import argparse
import geopandas as gpd
import fiona
import rasterio as rio
from rasterstats import zonal_stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap
from datetime import datetime
import glob
import pathlib


In [4]:
folder_root = os.getcwd()
print(folder_root)

folder_results =  os.path.join(folder_root, 'output')
print(folder_results)

e:\citarum\GeoAnalisisGEE
e:\citarum\GeoAnalisisGEE\output


In [5]:
desa_polygon_file = os.path.join(folder_root, 'input', 'vektor', 'batas_desa_citarum_48S.shp')
print(desa_polygon_file)
gdf = gpd.read_file(desa_polygon_file)

crs = gdf.crs
gdf['x_utm'] = gdf.centroid.x
gdf['y_utm'] = gdf.centroid.y

e:\citarum\GeoAnalisisGEE\input\vektor\batas_desa_citarum_48S.shp


In [6]:
list_raster_file = glob.glob(os.path.join(folder_root, 'input', 'raster', '*.tif'))
df_raster_file = pd.DataFrame(list_raster_file)
df_raster_file = df_raster_file.astype('string')
print(df_raster_file)
print(len(df_raster_file))
list(df_raster_file)
df_raster_file.rename(columns = {0:'filenamelong'}, inplace = True)

df_raster_file['filename'] = df_raster_file["filenamelong"].str.split("\\").str[-1]
df_raster_file['parameter'] = df_raster_file["filename"].str.split(".").str[0]
print(df_raster_file)


                                                   0
0  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...
1  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...
2  e:\citarum\GeoAnalisisGEE\input\raster\2020_et...
3  e:\citarum\GeoAnalisisGEE\input\raster\2020_fr...
4  e:\citarum\GeoAnalisisGEE\input\raster\2020_pa...
5  e:\citarum\GeoAnalisisGEE\input\raster\2020_pr...
6
                                        filenamelong                filename  \
0  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...            2020_dem.tif   
1  e:\citarum\GeoAnalisisGEE\input\raster\2020_de...  2020_depth_to_root.tif   
2  e:\citarum\GeoAnalisisGEE\input\raster\2020_et...            2020_eto.tif   
3  e:\citarum\GeoAnalisisGEE\input\raster\2020_fr...         2020_fractp.tif   
4  e:\citarum\GeoAnalisisGEE\input\raster\2020_pa...           2020_pawc.tif   
5  e:\citarum\GeoAnalisisGEE\input\raster\2020_pr...  2020_precipitation.tif   

            parameter  
0            2020_dem  
1  2020_depth_to_r

In [7]:
vars = []
zstats_merged = []

for ind in df_raster_file.index:
    raster_data = rio.open(df_raster_file['filenamelong'][ind], "r")  
    parameter = df_raster_file["parameter"][ind]
    print(parameter)
    profile = raster_data.profile
    transform = profile['transform']
    nodata = raster_data.nodata

    img = raster_data.read(1)
    img = img.astype('float32') 
    img[img==nodata] = np.nan
    zstats = zonal_stats(gdf, img, affine=transform, prefix= f'{parameter}_', nodata=nodata, stats='mean')
    zstats_merged.append(zstats) # zstats_merged is now a 2D list of lists [bands, shapes]
    vars.append(f'{parameter}_mean')
 

2020_dem
2020_depth_to_root
2020_eto
2020_fractp
2020_pawc
2020_precipitation


In [8]:
# Flip the dimensions using zip
zstats_merged_list = list(zip(*zstats_merged))
# Aggregate into a single list (dimension: shapes) containing bands
final_zstats = [{k: v for d in s for k, v in d.items()} for s in zstats_merged_list]
# Convert zonal statistic results to pandas dataframe
df_zstats = pd.DataFrame(final_zstats)

In [9]:
print(df_zstats)
pd.DataFrame(df_zstats).to_csv('sample.csv')    

      2020_dem_mean  2020_depth_to_root_mean  2020_eto_mean  2020_fractp_mean  \
0       1857.049733               700.000000     874.982169          0.242370   
1       1662.493644               806.024096    1579.000000          0.542684   
2       1629.247037              1500.000000    1523.854819          0.490222   
3        953.507028              1388.900588    1703.675641          0.471419   
4        583.723148              1369.361115    1804.776069          0.488760   
...             ...                      ...            ...               ...   
1285     775.182748               500.000000    1888.682750          0.438236   
1286     790.943335               500.000000    1886.235115          0.505137   
1287     819.765355               500.000000    1863.625903          0.578804   
1288     848.173172               500.000000    1831.240049          0.563120   
1289     216.835217               947.868718    1992.714877          0.839729   

      2020_pawc_mean  2020_

In [10]:
 # Merge zones and zonal statistics data frame
gdf = gdf.join(df_zstats)

# Clean the data
gdf = gdf.dropna()
print('Final input data shape: {}'.format(gdf.shape))

# Prepare datasets input
y = gdf[vars[0]].values.reshape((-1,1))
X = gdf[list(vars[1:])].values
coords = list(zip(gdf['x_utm'],gdf['y_utm']))

print('Independent variables: {}'.format(X.shape))
print('Dependent variable: {}'.format(y.shape))

X = (X - X.mean(axis=0)) / X.std(axis=0)
y = (y - y.mean(axis=0)) / y.std(axis=0)

# Calibrate MGWR model
print('Start MGWR ....')
start_time = time.time()
mgwr_selector = Sel_BW(coords, y, X, multi=True)
mgwr_bw = mgwr_selector.search(multi_bw_min=[2])
print('MGWR bandwidth: {}'.format(mgwr_bw))
mgwr_results = MGWR(coords, y, X, mgwr_selector).fit()
mgwr_results.summary()
elapsed_time = (time.time() - start_time)/60
print('MGWR complete. Elapsed time (minutes): '+str(elapsed_time))

print('Prepare MGWR results for mapping ...')
# Obtain t-vals filtered based on multiple testing correction
mgwr_filtered_t = mgwr_results.filter_tvals()



Final input data shape: (1256, 43)
Independent variables: (1256, 5)
Dependent variable: (1256, 1)
Start MGWR ....


c:\Users\fardh\anaconda3\envs\mgwr38-GEEMAP\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.54378e-18): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
c:\Users\fardh\anaconda3\envs\mgwr38-GEEMAP\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.46272e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
c:\Users\fardh\anaconda3\envs\mgwr38-GEEMAP\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.90591e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
c:\Users\fardh\anaconda3\envs\mgwr38-GEEMAP\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.51595e-17): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
c:\Users\fardh\anaconda3\envs\mgwr38-GEEMAP\lib\site-packages\spglm\iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.6974e-17): result may not be accurate.
  xtx_inv_xt = lin

Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

MGWR bandwidth: [10. 12. 16. 54. 12. 12.]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Model type                                                         Gaussian
Number of observations:                                                1256
Number of covariates:                                                     6

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                            145.408
Log-likelihood:                                                    -428.127
AIC:                                                                868.254
AICc:                                                               870.344
BIC:                                                              -8774.202
R2:                                                                   0.884
Adj. R2:                                                              0.884

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [12]:
print('Prepare MGWR results for mapping ...')
    # Obtain t-vals filtered based on multiple testing correction
mgwr_filtered_t = mgwr_results.filter_tvals()

    # Plot MGWR parameters
for i in range(1, 6):
    if i-1==0:
        param = 'mgwr_intercept'
    else:
        param = f'mgwr_b{i}'
    
    # Add MGWR parameters to GeoDataframe
    gdf[param] = mgwr_results.params[:,i-1]
    
    # Set color map
    vmin = gdf[param].min()
    vmax = gdf[param].max()
    cmap = plt.cm.seismic

    # Create scalar mappable for colorbar and stretch colormap across range of data values
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))

    # Plot
    fig, ax = plt.subplots(nrows=1, ncols=1)
    fig.set_size_inches(8, 6)
    ax.set_title(param, fontsize=10)
    gdf.plot(column=param,
        ax=ax,
        legend=True,
        cmap=sm.cmap, vmin=vmin, vmax=vmax, **{'edgecolor':'black', 'alpha':.65})
    if (mgwr_filtered_t[:,0] == 0).any(): #If there are insignificant parameters plot gray polygons over them
        gdf[mgwr_filtered_t[:,0] == 0].plot(color='lightgrey', ax=ax, **{'edgecolor':'black'})
    plt.savefig(folder_results + '\\png\\'+ param +'.png', dpi=100)
    plt.close()

print('Save output as vector shapefile ....')
with fiona.Env(OSR_WKT_FORMAT='WKT2_2018'):
    gdf.to_file(folder_results + '\\shp\\output1.shp')

print('{}-Done!'.format(datetime.now()))

Prepare MGWR results for mapping ...
Save output as vector shapefile ....


C:\Users\fardh\AppData\Local\Temp\ipykernel_22768\465978188.py:38: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(folder_results + '\\shp\\output1.shp')


2023-01-21 10:35:17.602888-Done!


In [ ]:
import ee
import geemap